# Linear Approach
This attempt focuses on using normalized centrality measures and the average centrality measures of each node's neighbors, a PCA and logistic regression for each of the languages

Starting by importing the right libraries:

In [1]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit, StratifiedGroupKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.ensemble import ExtraTreesClassifier
import statistics
import random
import warnings
from sklearn.exceptions import FitFailedWarning

warnings.filterwarnings("ignore", category=FitFailedWarning)
warnings.filterwarnings("ignore", category=UserWarning)

Next we read the train data:

In [2]:
data = pd.read_csv('../../data/normalized_augmented_train_1hop_allfeatures.csv')
data.head(5)

,language,sentence,vertex,n,degree,closeness,harmonic,betweeness,load,pagerank,...,current_flow_betweeness_nhop1_sum,percolation_nhop1_mean,percolation_nhop1_max,percolation_nhop1_sum,second_order_nhop1_mean,second_order_nhop1_max,second_order_nhop1_sum,laplacian_nhop1_mean,laplacian_nhop1_max,laplacian_nhop1_sum
0,Japanese,2,6,23,0.133038,-0.558160,-0.308023,-0.676962,-0.676962,0.409538,...,-0.961448,-0.861890,-0.537040,-0.961448,0.730613,1.021697,0.842049,-0.554857,0.758426,-0.578815
1,Japanese,2,4,23,-1.396899,-1.111953,-1.442852,-1.126101,-1.126101,-1.313638,...,-0.755507,-1.094197,-1.427690,-0.755507,0.748143,0.276860,0.553739,-0.554857,-1.127390,-0.552609
2,Japanese,2,2,23,1.662975,0.111311,0.741550,0.135764,0.135764,1.885752,...,-1.223610,-0.820895,-0.232344,-1.223610,0.230733,0.352194,0.378021,-0.732103,-0.184482,-0.906393
3,Japanese,2,23,23,-1.396899,-0.618060,-0.907167,-1.126101,-1.126101,-1.449223,...,-0.221405,-0.055651,-0.537040,-0.221405,-0.098853,-0.500142,0.123792,1.572094,0.758426,0.652876
4,Japanese,2,20,23,0.133038,0.812630,0.880961,0.413803,0.413803,-0.074579,...,1.076574,1.119547,1.478643,1.076574,-0.906266,-0.500142,-0.819755,1.837963,1.229880,2.133527


Now we will focus only in one language, opting to go for `Polish`:

In [3]:
polish_data = data[data.language == 'Polish'].sample(frac=1, random_state=1).copy()
polish_data.head(5)

,language,sentence,vertex,n,degree,closeness,harmonic,betweeness,load,pagerank,...,current_flow_betweeness_nhop1_sum,percolation_nhop1_mean,percolation_nhop1_max,percolation_nhop1_sum,second_order_nhop1_mean,second_order_nhop1_max,second_order_nhop1_sum,laplacian_nhop1_mean,laplacian_nhop1_max,laplacian_nhop1_sum
126122,Polish,46,2,23,-0.983415,-1.112357,-1.140267,-0.862200,-0.862200,-0.946343,...,-0.673061,-0.659982,-1.144528,-0.673061,0.733398,0.261813,0.570124,-0.037514,-0.671478,-0.291875
130297,Polish,567,25,35,-0.931149,-1.425134,-1.327955,-0.808350,-0.808350,-0.898689,...,-0.837453,-0.868841,-1.219894,-0.837453,1.273974,0.900006,0.774337,-0.056030,-0.548010,-0.310756
128519,Polish,357,9,10,-0.816497,-0.465846,-0.528847,-0.836117,-0.836117,-0.841449,...,0.586736,1.267236,0.274625,0.586736,-1.190242,-1.264715,-0.493920,1.424305,0.857505,1.044150
132310,Polish,825,10,15,2.083023,0.453251,1.350470,1.156678,1.156678,2.162341,...,-1.650223,-1.204552,0.113984,-1.650223,0.788028,0.734528,1.114068,-1.159880,-1.236159,-1.813025
129666,Polish,490,26,27,0.068199,0.728343,0.712585,0.420129,0.420129,-0.047339,...,1.228535,0.913466,1.084363,1.228535,-0.607010,-0.091420,-0.535908,0.897904,1.192677,1.094490


Now we can use pca to reduce the centrality dimensions:

In [4]:
pca = PCA(n_components=7)
feature_cols = ['degree', 'closeness','harmonic', 'betweeness', 'load', 'pagerank', 'eigenvector', 'katz',
       'information', 'current_flow_betweeness', 'percolation', 'second_order',
       'laplacian', 'degree_nhop1_mean', 'degree_nhop1_max',
       'degree_nhop1_sum', 'closeness_nhop1_mean', 'closeness_nhop1_max',
       'closeness_nhop1_sum', 'harmonic_nhop1_mean', 'harmonic_nhop1_max',
       'harmonic_nhop1_sum', 'betweeness_nhop1_mean', 'betweeness_nhop1_max',
       'betweeness_nhop1_sum', 'load_nhop1_mean', 'load_nhop1_max', 'load_nhop1_sum', 'pagerank_nhop1_mean', 'pagerank_nhop1_max',
       'pagerank_nhop1_sum', 'eigenvector_nhop1_mean', 'eigenvector_nhop1_max',
       'eigenvector_nhop1_sum', 'katz_nhop1_mean', 'katz_nhop1_max',
       'katz_nhop1_sum', 'information_nhop1_mean', 'information_nhop1_max',
       'information_nhop1_sum', 'current_flow_betweeness_nhop1_mean',
       'current_flow_betweeness_nhop1_max','current_flow_betweeness_nhop1_sum', 'percolation_nhop1_mean',
       'percolation_nhop1_max', 'percolation_nhop1_sum','second_order_nhop1_mean', 'second_order_nhop1_max',
       'second_order_nhop1_sum', 'laplacian_nhop1_mean', 'laplacian_nhop1_max',
       'laplacian_nhop1_sum']
X_pca = pca.fit_transform(polish_data[feature_cols])

The explained variance reaches almost 100% by just using 7 principal components

In [5]:
sum(pca.explained_variance_ratio_)

np.float64(0.9844359775087458)

Now we can split the data in train and test in a way to keep the same sentences (sentence level and node level of sentence) in the same group. Each sentence and its nodes will only be found in train or validation set

In [6]:
X = polish_data.copy()#.drop(['is_root','language','n'], axis=1).copy()
y = polish_data['is_root'].copy()

groups = polish_data['sentence']

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(gss.split(X, y, groups=groups))

X_train, X_val= X.iloc[train_idx], X.iloc[val_idx]
y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

Now we also define the cross validation strategy. We also split the training data in 5 different folds ensuring that data of of the same sentence can not be found in different folds:

In [7]:
cv = StratifiedGroupKFold(n_splits=5)
groups = X_train['sentence']

Next we can define the pipeline of pca and logistic regression model parameters that we want to try:

In [8]:
# Define columns
feature_cols = ['sentence', 'degree', 'closeness','harmonic', 'betweeness', 'load', 'pagerank', 'eigenvector', 'katz',
       'information', 'current_flow_betweeness', 'percolation', 'second_order',
       'laplacian', 'degree_nhop1_mean', 'degree_nhop1_max',
       'degree_nhop1_sum', 'closeness_nhop1_mean', 'closeness_nhop1_max',
       'closeness_nhop1_sum', 'harmonic_nhop1_mean', 'harmonic_nhop1_max',
       'harmonic_nhop1_sum', 'betweeness_nhop1_mean', 'betweeness_nhop1_max',
       'betweeness_nhop1_sum', 'load_nhop1_mean', 'load_nhop1_max', 'load_nhop1_sum', 'pagerank_nhop1_mean', 'pagerank_nhop1_max',
       'pagerank_nhop1_sum', 'eigenvector_nhop1_mean', 'eigenvector_nhop1_max',
       'eigenvector_nhop1_sum', 'katz_nhop1_mean', 'katz_nhop1_max',
       'katz_nhop1_sum', 'information_nhop1_mean', 'information_nhop1_max',
       'information_nhop1_sum', 'current_flow_betweeness_nhop1_mean',
       'current_flow_betweeness_nhop1_max','current_flow_betweeness_nhop1_sum', 'percolation_nhop1_mean',
       'percolation_nhop1_max', 'percolation_nhop1_sum','second_order_nhop1_mean', 'second_order_nhop1_max',
       'second_order_nhop1_sum', 'laplacian_nhop1_mean', 'laplacian_nhop1_max',
       'laplacian_nhop1_sum']

pca_columns = X_train[feature_cols].columns.difference(['sentence'])
passthrough_columns = ['sentence']

# We want the sentence column to passthrough and not be transformed since it is used for the scoring
preprocessor = ColumnTransformer([
     ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=7))
    ]), pca_columns),
    ('passthrough', 'passthrough', passthrough_columns)
])

# Full pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=2, n_jobs=7))
])

param_grid = {
    'classifier__penalty': ['l1', 'l2', 'elasticnet', None],
    'classifier__C': [0.01, 0.1, 1, 10, 100],
    'classifier__solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    'classifier__l1_ratio': [0, 0.5, 1],
    'classifier__fit_intercept':[True,False],
    'classifier__warm_start': [True,False],
    'classifier__max_iter': [100, 250, 500, 1000, 2500, 5000]
}


We use a custom scoring method that picks one root per sentence. For each sentence, it chooses the node with the highest chance of being the root (class 1). The final score is just the percentage of sentences where we picked the correct root.:

In [9]:
def root_prediction_score(estimator, X, y_true):
    """
    Scoring function that extracts sentence IDs from X and computes
    root prediction accuracy per sentence.
    """
    
    sentence_ids = X['sentence'].values
    X_features = X.copy()
    # Predict probabilities
    probs = estimator.predict_proba(X_features)[:, 1]
    # Build DataFrame for groupby
    df_pred = pd.DataFrame({
        'sentence': sentence_ids,
        'is_root': y_true,
        'root_prob': probs
    })

    predicted_roots = df_pred.loc[df_pred.groupby('sentence')['root_prob'].idxmax()]
    accuracy = float((predicted_roots['is_root'] == 1).mean())
    return accuracy

root_scorer = make_scorer(root_prediction_score, greater_is_better=True)


Now we can do the gridsearch, combining defined parameters, the custom scoring function, the cross validation strategy (that ensures no data leakage):

In [10]:

grid_search = RandomizedSearchCV(n_iter=500,
    estimator=pipeline,
    param_distributions=param_grid,
    cv=cv,
    scoring=root_prediction_score,
    n_jobs=7,
    random_state=2
)

grid_search.fit(X_train[feature_cols], y_train, groups=groups)

print("Best params:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best params: {'classifier__warm_start': False, 'classifier__solver': 'lbfgs', 'classifier__penalty': None, 'classifier__max_iter': 100, 'classifier__l1_ratio': 1, 'classifier__fit_intercept': True, 'classifier__C': 0.01}
Best score: 0.28500000000000003


We will use the best estimator found to see the performance in the validation dataset:

In [11]:
best_estimator = grid_search.best_estimator_
root_prediction_score(best_estimator, X_val[feature_cols], y_val)

0.35

### Extending all languages
Next we create a script that follows the previous steps for each of the 21 languages. As a result we will have one logistic regression model for each language. To predict the test data, we will use the particular logistic regression depending on the language of the row:

In [ ]:
score = []
best_models = {}
random.seed(42)
for language in data.language.unique():
    X = data[data.language == language].sample(frac=1, random_state=1).copy()
    y = X['is_root'].copy()

    groups = X['sentence']

    gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    train_idx, val_idx = next(gss.split(X, y, groups=groups))

    X_train, X_val= X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    cv = StratifiedGroupKFold(n_splits=5)
    groups = X_train['sentence']
    
    feature_cols = [
    'sentence', 'degree', 'closeness', 'harmonic', 'betweeness', 'load', 'pagerank',
    'eigenvector', 'katz', 'information', 'current_flow_betweeness',
    'percolation', 'second_order', 'laplacian'
    ]
    pca_columns = X_train[feature_cols].columns.difference(['sentence'])
    passthrough_columns = ['sentence']

    preprocessor = ColumnTransformer([
        ('pca_pipeline', Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA())
        ]), pca_columns),
        ('passthrough', 'passthrough', passthrough_columns)
    ])

    # Final pipeline of pca and classifier
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(random_state=2, n_jobs=7))
    ])

    param_grid = {
        'preprocessor__pca_pipeline__pca__n_components': [5,6,7,8,9],
        'classifier__penalty': ['l1', 'l2', 'elasticnet', None],
        'classifier__C': [0.01, 0.1, 1, 10, 100],
        'classifier__solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
        'classifier__l1_ratio': [0, 0.5, 1],
        'classifier__fit_intercept':[True,False],
        'classifier__warm_start': [True,False],
        'classifier__max_iter': [50, 100, 250, 500, 750, 1000, 2000]
    }

    grid_search = RandomizedSearchCV(n_iter=500,
    estimator=pipeline,
    param_distributions=param_grid,
    cv=cv,
    scoring=root_prediction_score,
    n_jobs=7,
    # We want to get random combination of params for each different language
    random_state=random.randint(1, 1000)
    )

    grid_search.fit(X_train[feature_cols], y_train, groups=groups)

    best_params = grid_search.best_params_
    
    current_score = root_prediction_score(grid_search.best_estimator_, X_val[feature_cols], y_val)
    score.append(current_score)
    print(language)
    print('Grid search score: ' + str(grid_search.best_score_))
    print('Validation score: ' + str(current_score))
    print(best_params)
    print('---------------------------------------')
    
    # Rebuild the whole pipeline using the best params
    final_preprocessor = ColumnTransformer([
        ('pca_pipeline', Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA(n_components=best_params['preprocessor__pca_pipeline__pca__n_components']))
        ]), pca_columns),
        ('passthrough', 'passthrough', passthrough_columns)
    ])

    # Handle optional params
    classifier_params = {
        key.split("classifier__")[1]: value
        for key, value in best_params.items()
        if key.startswith("classifier__")
    }

    final_model = Pipeline([
        ('preprocessor', final_preprocessor),
        ('classifier', LogisticRegression(
            **classifier_params,
            n_jobs=7,
            random_state=2
        ))
    ])


    final_model.fit(X[feature_cols], y)
    best_models[language] = final_model

Japanese
Grid search score: 0.095
Validation score: 0.09
{'preprocessor__pca_pipeline__pca__n_components': 5, 'classifier__warm_start': True, 'classifier__solver': 'newton-cg', 'classifier__penalty': None, 'classifier__max_iter': 500, 'classifier__l1_ratio': 1, 'classifier__fit_intercept': True, 'classifier__C': 0.1}
---------------------------------------
Finnish
Grid search score: 0.355
Validation score: 0.35
{'preprocessor__pca_pipeline__pca__n_components': 9, 'classifier__warm_start': True, 'classifier__solver': 'lbfgs', 'classifier__penalty': None, 'classifier__max_iter': 250, 'classifier__l1_ratio': 0.5, 'classifier__fit_intercept': True, 'classifier__C': 0.1}
---------------------------------------
Galician
Grid search score: 0.29250000000000004
Validation score: 0.32
{'preprocessor__pca_pipeline__pca__n_components': 9, 'classifier__warm_start': True, 'classifier__solver': 'newton-cholesky', 'classifier__penalty': None, 'classifier__max_iter': 500, 'classifier__l1_ratio': 1, 'cl

Printing the average score for the whole validation dataset:

In [49]:
statistics.mean(score)

0.2957142857142857